This script loads pre-trained word embeddings (GloVe embeddings) into a frozen Keras Embedding layer, and uses it to train a text classification model on the 20 Newsgroup dataset (classification of newsgroup messages into 20 different categories).

Wiki word vectors can be found at: https://fasttext.cc/docs/en/pretrained-vectors.html

Yelp reviews can be found at: https://www.kaggle.com/yelp-dataset/yelp-dataset

In [1]:
import os
import sys
import numpy as np
import multiprocessing
import tensorflow as tf
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer
from fastText import load_model

/data/joydeep/pytorchfasttext/conda_venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
BASE_DIR = ''
# TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
FT_MODEL = 'wiki.simple.bin'

In [3]:
%%time
# first, build index mapping words in the embeddings set
# to their embedding vector
# Use this in case you are using a vec file.

print('Indexing word vectors.')

filename, file_extension = os.path.splitext(FT_MODEL)

if file_extension == '.vec':
    embeddings_index = {}
    with open(os.path.join(FT_MODEL)) as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
            
    print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 83.7 µs


In [4]:
def clean_str(string):
    """
    Tokenization/string cleaning.
    Original from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    
    return string.strip().lower()

Prepare text samples and their labels

In [5]:
df = pd.read_csv('yelp_review.csv')

In [6]:
%%time
df['text'] = df['text'].apply(clean_str)
texts = df.text.values
labels = df.stars.values
texts = texts[:MAX_NUM_WORDS]
labels = labels[:MAX_NUM_WORDS]

CPU times: user 7min, sys: 2.49 s, total: 7min 3s
Wall time: 7min 3s


In [7]:
df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,super simple place but amazing nonetheless it ...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,lester 's is located in a beautiful neighborho...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,love coming here yes the place always needs th...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,had their chocolate almond croissant and it wa...,0,0,0


In [8]:
print('Found %s texts.' % len(texts))

Found 20000 texts.


In [9]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 43675 unique tokens.


In [11]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (20000, 1000)
Shape of label tensor: (20000, 6)


In [13]:
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (20000, 1000)
Shape of label tensor: (20000, 6)


In [14]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [15]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [16]:
print('Preparing embedding matrix.')

# load the fasttext model
f = load_model(FT_MODEL)

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
#     embedding_vector = embeddings_index.get(word)
    embedding_vector = f.get_word_vector(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [17]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [18]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(6, activation='softmax')(x)

Training model.


In [19]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001, decay=1e-6),
              metrics=['acc'])

In [20]:
%%time
model.fit(x_train, y_train,
          batch_size=128,
          shuffle=True,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 16000 samples, validate on 4000 samples
Epoch 1/10
16000/16000 [==============================] - 294s 18ms/step - loss: 1.4900 - acc: 0.3701 - val_loss: 1.4188 - val_acc: 0.3925
Epoch 2/10
16000/16000 [==============================] - 288s 18ms/step - loss: 1.4039 - acc: 0.3958 - val_loss: 1.3503 - val_acc: 0.4175
Epoch 3/10
16000/16000 [==============================] - 284s 18ms/step - loss: 1.3113 - acc: 0.4308 - val_loss: 1.2583 - val_acc: 0.4535
Epoch 4/10
16000/16000 [==============================] - 287s 18ms/step - loss: 1.1848 - acc: 0.4899 - val_loss: 1.1642 - val_acc: 0.4890
Epoch 5/10
16000/16000 [==============================] - 296s 19ms/step - loss: 1.1023 - acc: 0.5309 - val_loss: 1.1173 - val_acc: 0.5135
Epoch 6/10
16000/16000 [==============================] - 284s 18ms/step - loss: 1.0464 - acc: 0.5592 - val_loss: 1.0872 - val_acc: 0.5288
Epoch 7/10
16000/16000 [==============================] - 284s 18ms/step - loss: 0.9978 - acc: 0.5830 - val_loss: 1.0

In [21]:
scores = model.evaluate(x_val, y_val)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

4000/4000 [==============================] - 31s 8ms/step

acc: 55.93%


In [22]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          192128    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048     
__________